In [1]:
import configparser
import psycopg2
import time
from sql_queries import copy_table_queries, insert_table_queries
from create_tables import create_tables, drop_tables

In [2]:
def load_staging_tables(cur, conn):
    for query in copy_table_queries:
        print(query)
        cur.execute(query)
        conn.commit()

def insert_tables(cur, conn):
    for query in insert_table_queries:
        print(query)
        cur.execute(query)
        conn.commit()

In [3]:
config = configparser.ConfigParser()
config.read('dwh.cfg')

conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(*config['CLUSTER'].values()))
cur = conn.cursor()

print("1. Dropping tables...")
start_time = time.time()
drop_tables(cur, conn)
print("--- It took %s seconds ---" % (time.time() - start_time))

print("2. Creating tables...")
start_time = time.time()
create_tables(cur, conn)
print("--- It took %s seconds ---" % (time.time() - start_time))

#print("3. Loading staging tables, it may take time...")
#start_time = time.time()
#load_staging_tables(cur, conn)
#print("--- It took %s seconds ---" % (time.time() - start_time))

print("4. Loading final tables...")
start_time = time.time()
insert_tables(cur, conn)
print("--- It took %s seconds ---" % (time.time() - start_time))

conn.close()

1. Dropping tables...
--- It took 3.845492362976074 seconds ---
2. Creating tables...
--- It took 6.104329586029053 seconds ---
4. Loading final tables...

        INSERT INTO time (start_time, hour, day, week, month, year, weekday)
        SELECT
            TIMESTAMP 'epoch' + ts/1000 *INTERVAL '1 second' as start_time,
            date_part(h, start_time) as hour,
            date_part(d, start_time) as day,
            date_part(w, start_time) as week,
            date_part(m, start_time) as month,
            date_part(y, start_time) as year,
            date_part(dow, start_time) as weekday
        FROM staging_events
        WHERE ts IS NOT NULL

--- It took 1.0767579078674316 seconds ---
